# Predict customer retention by deep learning

# 1. Classify structured data with feature columns

## Dataset

In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import model_selection,pipeline,compose,preprocessing
from tensorflow import feature_column
from tensorflow.keras import layers
# OOP python for feature engineering
from sklearn.base import BaseEstimator, TransformerMixin

In [67]:
path='https://raw.githubusercontent.com/rstudio/keras-customer-churn/master/data/WA_Fn-UseC_-Telco-Customer-Churn.csv'
dataframe=pd.read_csv(path)
dataframe.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
dataframe.shape

(7043, 21)

In [4]:
dataframe.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


## Create target variable

In [68]:
# In the original dataset "4" indicates the pet was not adopted. ( No need in this case)
#dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)
dataframe.rename(columns = {'Churn': 'target'}, inplace=True)
dataframe['target'] = np.where(dataframe['target']=="Yes", 1, 0)
# Drop un-used columns.
dataframe = dataframe.drop(columns=['customerID'])
# Convert total charge from object to float:
dataframe["TotalCharges"] = dataframe["TotalCharges"].replace(' ', np.nan).astype(float)
dataframe.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [69]:
train, test = model_selection.train_test_split(dataframe, test_size=0.2)
train, val = model_selection.train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

4507 train examples
1127 validation examples
1409 test examples


Next, we will wrap the dataframes with tf.data. This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [70]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [71]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size) # need suffle train ?
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [72]:
print(len(list(train_ds))) #902 batchs, each batch 5 sample --> 4507 sample
print(len(list(val_ds)))
print(len(list(test_ds)))

902
226
282


In [73]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of MonthlyCharges:', train_features['MonthlyCharges'])
print('A batch of targets:', label_batch )

Every feature: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']
A batch of MonthlyCharges: tf.Tensor([ 74.65  49.95 115.8   95.95  74.9 ], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([1 1 0 0 1], shape=(5,), dtype=int32)


Demonstrate several types of feature columns

In [74]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]
example_batch # contain only x train, not include y train

{'gender': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Male', b'Male', b'Male', b'Male', b'Female'], dtype=object)>,
 'SeniorCitizen': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0], dtype=int64)>,
 'Partner': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Yes', b'No', b'Yes', b'Yes', b'Yes'], dtype=object)>,
 'Dependents': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Yes', b'No', b'Yes', b'No', b'No'], dtype=object)>,
 'tenure': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 7, 21, 63, 56, 33], dtype=int64)>,
 'PhoneService': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Yes', b'Yes', b'Yes', b'No', b'No'], dtype=object)>,
 'MultipleLines': <tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'No', b'No', b'No', b'No phone service', b'No phone service'],
       dtype=object)>,
 'InternetService': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'No', b'DSL', b'No', b'DSL', b'DSL'], dtype=object)>,
 'OnlineSecurity': <tf.Tensor: s

In [75]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

### Numeric columns
The output of a feature column becomes the input to the model (using the demo function defined above, we will be able to see exactly how each column from the dataframe is transformed). A [numeric column](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) is the simplest type of column. It is used to represent real valued features. When using this column, your model will receive the column value from the dataframe unchanged.

In [76]:
example_batch['MonthlyCharges']

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([20.65, 61.65, 19.7 , 54.05, 59.55])>

In [77]:
MonthlyCharges = feature_column.numeric_column('MonthlyCharges') # feature column from tensorflow
MonthlyCharges

NumericColumn(key='MonthlyCharges', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [78]:
demo(MonthlyCharges) # show one sample from feature column


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[20.65]
 [61.65]
 [19.7 ]
 [54.05]
 [59.55]]


### Bucketized columns
Often, you don't want to feed a number directly into the model, but instead split its value into different categories based on numerical ranges. Consider raw data that represents a person's age. Instead of representing age as a numeric column, we could split the age into several buckets using a [bucketized column](https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column). Notice the one-hot values below describe which age range each row matches.

In [79]:
example_batch['MonthlyCharges']

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([20.65, 61.65, 19.7 , 54.05, 59.55])>

In [80]:
MonthlyCharges = feature_column.numeric_column('MonthlyCharges')
MonthlyCharges_buckets = feature_column.bucketized_column(MonthlyCharges, boundaries=[1, 50, 200])
MonthlyCharges_buckets

BucketizedColumn(source_column=NumericColumn(key='MonthlyCharges', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(1, 50, 200))

In [81]:
demo(MonthlyCharges_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


### Categorical columns
In this dataset, Type is represented as a string (e.g. 'Dog', or 'Cat'). We cannot feed strings directly to a model. Instead, we must first map them to numeric values. The categorical vocabulary columns provide a way to represent strings as a one-hot vector (much like you have seen above with age buckets). The vocabulary can be passed as a list using [categorical_column_with_vocabulary_list](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list), or loaded from a file using [categorical_column_with_vocabulary_file](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_file).

In [82]:
example_batch['PhoneService']

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Yes', b'Yes', b'Yes', b'No', b'No'], dtype=object)>

In [83]:
PhoneService_type = feature_column.categorical_column_with_vocabulary_list(
      'PhoneService', dataframe.PhoneService.unique())

PhoneService_type

VocabularyListCategoricalColumn(key='PhoneService', vocabulary_list=('No', 'Yes'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

In [84]:
PhoneService_type_one_hot = feature_column.indicator_column(PhoneService_type)
demo(PhoneService_type_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


### Embedding columns
Suppose instead of having just a few possible strings, we have thousands (or more) values per category. For a number of reasons, as the number of categories grow large, it becomes infeasible to train a neural network using one-hot encodings. We can use an embedding column to overcome this limitation. Instead of representing the data as a one-hot vector of many dimensions, an [embedding column](https://www.tensorflow.org/api_docs/python/tf/feature_column/embedding_column) represents that data as a lower-dimensional, dense vector in which each cell can contain any number, not just 0 or 1. The size of the embedding (8, in the example below) is a parameter that must be tuned.

Key point: using an embedding column is best when a categorical column has many possible values. We are using one here for demonstration purposes, so you have a complete example you can modify for a different dataset in the future.

In [85]:
example_batch['MultipleLines']

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'No', b'No', b'No', b'No phone service', b'No phone service'],
      dtype=object)>

In [86]:
# Notice the input to the embedding column is the categorical column
# we previously created
MultipleLines = feature_column.categorical_column_with_vocabulary_list(
      'MultipleLines', dataframe.MultipleLines.unique())
MultipleLines_embedding = feature_column.embedding_column(MultipleLines, dimension=4)
demo(MultipleLines_embedding)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[-0.30681494  0.27269313  0.38547808 -0.03851676]
 [-0.30681494  0.27269313  0.38547808 -0.03851676]
 [-0.30681494  0.27269313  0.38547808 -0.03851676]
 [ 0.5267758   0.04663187 -0.0917053  -0.85766596]
 [ 0.5267758   0.04663187 -0.0917053  -0.85766596]]


### Hashed feature columns

Another way to represent a categorical column with a large number of values is to use a [categorical_column_with_hash_bucket](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_hash_bucket). This feature column calculates a hash value of the input, then selects one of the `hash_bucket_size` buckets to encode a string. When using this column, you do not need to provide the vocabulary, and you can choose to make the number of hash_buckets significantly smaller than the number of actual categories to save space.

Key point: An important downside of this technique is that there may be collisions in which different strings are mapped to the same bucket. In practice, this can work well for some datasets regardless.

In [87]:
MultipleLines_hashed = feature_column.categorical_column_with_hash_bucket(
      'MultipleLines', hash_bucket_size=10)
demo(feature_column.indicator_column(MultipleLines_hashed))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


### Crossed feature columns
Combining features into a single feature, better known as [feature crosses](https://developers.google.com/machine-learning/glossary/#feature_cross), enables a model to learn separate weights for each combination of features. Here, we will create a new feature that is the cross of Age and Type. Note that `crossed_column` does not build the full table of all possible combinations (which could be very large). Instead, it is backed by a `hashed_column`, so you can choose how large the table is.

In [88]:
demo(MonthlyCharges_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [89]:
PhoneService_type_one_hot = feature_column.indicator_column(PhoneService_type)
demo(PhoneService_type_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [90]:
#crossed_feature = feature_column.crossed_column([MonthlyCharges_buckets, PhoneService_type], hash_bucket_size=10)
#demo(feature_column.indicator_column(crossed_feature))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



OverflowError: Python int too large to convert to C long

## Choose which columns to use
We have seen how to use several types of feature columns. Now we will use them to train a model. The goal of this tutorial is to show you the complete code (e.g. mechanics) needed to work with feature columns. We have selected a few columns to train our model below arbitrarily.

Key point: If your aim is to build an accurate model, try a larger dataset of your own, and think carefully about which features are the most meaningful to include, and how they should be represented.

In [91]:
#train.head()

dataframe_remove_target = dataframe.iloc[:,:-1]
dataframe_remove_target.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [92]:
numerical_column =list(dataframe_remove_target.select_dtypes('number').columns)

In [93]:
feature_columns = []
# numeric cols
for header in numerical_column:
  feature_columns.append(feature_column.numeric_column(header))

In [94]:
# bucketized cols (bin numeric column, suggest for monthly charge and total charge)
#age = feature_column.numeric_column('Age')
#age_buckets = feature_column.bucketized_column(age, boundaries=[1, 2, 3, 4, 5])
#feature_columns.append(age_buckets)

In [95]:
# indicator_columns (for category)
categorical_column = list(dataframe_remove_target.select_dtypes('object').columns) # Remove target column
for col_name in categorical_column:
  categorical_column = feature_column.categorical_column_with_vocabulary_list( col_name, dataframe_remove_target[col_name].unique()) # Define One hot encoder
  indicator_column = feature_column.indicator_column(categorical_column) # convert to one hot
  feature_columns.append(indicator_column)

In [96]:
# embedding columns (for category with a lot of unique item, not apply here)
#breed1 = feature_column.categorical_column_with_vocabulary_list(
#      'Breed1', dataframe.Breed1.unique())
#breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
#feature_columns.append(breed1_embedding)

In [97]:
# crossed columns (no need here)
#age_type_feature = feature_column.crossed_column([age_buckets, animal_type], #hash_bucket_size=100)
#feature_columns.append(feature_column.indicator_column(age_type_feature))

In [98]:
feature_columns

[NumericColumn(key='SeniorCitizen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='MonthlyCharges', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='TotalCharges', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Female', 'Male'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Partner', vocabulary_list=('Yes', 'No'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Dependents', vocabulary_list=('No', 'Yes'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalC

Now that we have defined our feature columns, we will use a DenseFeatures layer to input them to our Keras model.

In [99]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.

In [100]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## Create, compile, and train the model

In [106]:
train.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
target                int32
dtype: object

In [101]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [102]:
#import graphviz
#import pydot
# rankdir='LR' is used to make the graph horizontal.
#tf.keras.utils.plot_model(model, show_shapes=True, #rankdir="LR"
#)

In [103]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

141/141 [==============================] - 2s 17ms/step - loss: nan - accuracy: 0.7304 - val_loss: nan - val_accuracy: 0.7347
Epoch 2/10
141/141 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.7337 - val_loss: nan - val_accuracy: 0.7347
Epoch 3/10
141/141 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.7337 - val_loss: nan - val_accuracy: 0.7347
Epoch 4/10
141/141 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.7337 - 

In [104]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

45/45 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7374
Accuracy 0.7374024391174316


In [105]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_11 (DenseFeat multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  5888      
_________________________________________________________________
dense_7 (Dense)              multiple                  16512     
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                  129       
Total params: 22,529
Trainable params: 22,529
Non-trainable params: 0
_________________________________________________________________


Key point: You will typically see best results with deep learning with much larger and more complex datasets. When working with a small dataset like this one, we recommend using a decision tree or random forest as a strong baseline. The goal of this tutorial is not to train an accurate model, but to demonstrate the mechanics of working with structured data, so you have code to use as a starting point when working with your own datasets in the future.